In [1]:
import re
import pickle
from tqdm.notebook import tqdm as tqdm  # progress bar for x in tqdm(range(100))

# selenium and beauitfulsoup
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions

from bs4 import BeautifulSoup
import requests
import time
import os
import sys
sys.setrecursionlimit(50000) # because pickle says max recurssion limit reached

In [2]:
url = 'http://nomadlist.com/userApi.php?action=login_by_email&hash=caea33d5d8b2cd46ee4f3054a79172806dfcea43'

In [278]:
# path to the chromedriver executable
chromedriver = "/Applications/chromedriver"
os.environ["webdriver.chrome.driver"] = chromedriver

driver = webdriver.Chrome(chromedriver)
driver.get(url)

# Wait for page to load
time.sleep(5)

In [279]:
# infinite scroll
for i in range(20):
    
    # Scroll down to bottom
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    
    soup = BeautifulSoup(driver.page_source, 'html.parser')

In [280]:
soup = BeautifulSoup(driver.page_source, 'html.parser')

In [281]:
# get the links to the cities
cities = []
for li in soup.find_all('li'):
    cities.append((li.get('data-slug')))
    
# append to pickle
with open('nomadlist_cities.pkl', 'wb') as picklefile:
    pickle.dump(cities, picklefile)

print(len(cities))

# Get Information For Each City

In [283]:
def remove_punctuation(doc): # left out period
    return re.sub('[^A-Za-z0-9\.]+', ' ', doc).strip()
    #return re.sub('[%s]' % re.escape('"#$%&\()*+-/:;<=>@[\\]^_`{|}~”“—.,!?'), ' ', doc).strip()

In [284]:
def remove_all_punctuation(doc):
    return re.sub('[^A-Za-z0-9]+', ' ', doc).strip()
    #return re.sub('[%s]' % re.escape('"#$%&\()*+-/:;<=>@[\\]^_`{|}~”“—.,!?'), ' ', doc).strip()

In [285]:
nomadlist = pd.read_csv('nomadlist.csv')
nomadlist

,City,Country,Internet Speed (Mbps),Nomad Cost ($ monthly),Fun (out of 5),Safety (out of 5),link
0,Bangkok,Thailand,24,"1,522",4.0,4.00,bangkok
1,Buenos Aires,Argentina,18,"1,028",4.0,2.68,buenos-aires
2,Chiang Mai,Thailand,23,"1,126",4.0,4.09,chiang-mai
3,Taipei,Taiwan,20,"2,120",3.0,5.00,taipei
4,Canggu,Indonesia,20,"1,386",4.0,4.05,canggu
...,...,...,...,...,...,...,...
1327,Mogadishu,Somalia,2,"2,708",2.0,0.25,mogadishu
1328,Tangshan,China,2,"2,253",2.0,4.18,tangshan
1329,Ashgabat,Turkmenistan,2,"3,737",2.0,4.00,ashgabat
1330,N'Djamena,Chad,0,"3,964",2.0,2.50,ndjamena


In [286]:
city_info_dict = {}
people_info_dict = {}
for city in tqdm(nomadlist['link']):
    try:
        # scores--------------------------------------------------------------
        response = requests.get('http://nomadlist.com/' + city)
        page = response.text
        soup = BeautifulSoup(page, "html.parser")
        
        current_city_info = {}
        for key in soup.find_all(class_='key'):
            # remove the punctuation and emojies
            current_city_info[remove_punctuation(key.text)]  = remove_punctuation(key.findNextSibling().text)
        
        # people that have been to that city ----------------------------------
        people = []
        for person in soup.find(class_='people-here-now').find_all('a'):
            people.append((person.get('href')))
            
            
        # append to dictionaries if both scrapes work---------------------------
        city_info_dict[city] = current_city_info
        people_info_dict[city] = people
        
    except Exception:
        print(Exception)
        print(response.status_code)
        print('For ', city)

In [287]:
# save the dictionaries to a pickle 
with open(f'city_info_dict.pkl', 'wb') as picklefile:
    pickle.dump(city_info_dict, picklefile)
    
with open(f'people_info_dict.pkl', 'wb') as picklefile:
    pickle.dump(people_info_dict, picklefile)

# Open Raw Pickles

In [361]:
# open links
with open("city_info_dict.pkl", 'rb') as picklefile: 
    city_info_dict = pickle.load(picklefile)

# open links
with open("people_info_dict.pkl", 'rb') as picklefile: 
    people_info_dict = pickle.load(picklefile)

So the features are not consistent, that's why we end up with over 600+ columns.   
Going to drop columns with over 100 nulls.

In [362]:
city_info_df = pd.DataFrame(city_info_dict).transpose().reset_index().dropna(axis=1, thresh=100)
city_info_df.columns

Index(['index', 'Nomad Score', 'Cost', 'Internet', 'Fun', 'Temperature now',
       'Humidity now', 'Air quality now', 'Safety', 'Quality of life',
       'People density', 'Walkability', 'Peace', 'Traffic safety', 'Hospitals',
       'Happiness', 'Nightlife', 'Free WiFi in city', 'Places to work from',
       'A C or heating', 'Friendly to foreigners', 'English speaking',
       'Freedom of speech', 'Racial tolerance', 'Female friendly',
       'LGBT friendly', 'Startup Score', 'Region', 'Country',
       'Average trip duration', 'Internet speed avg', 'Weather',
       'Air quality annual avg', 'Power', 'Best taxi app',
       'Best wireless carrier', 'Tipping', 'Cashless society',
       'Best coworking space', 'Best coffee place', 'Best 24 7 coffee place',
       'Safe tap water', 'Return rate', 'Population', 'Gender ratio',
       'Religious government', 'Online electronics shop', 'Apartment listings',
       'Best short haul air carrier', 'Best int l air carrier',
       'Best hos

# Creating a clean DataFrame

## city

In [364]:
# create a df and add the city name
city_info_final = city_info_df[['index']]
city_info_final.columns= ['city']

In [365]:
city_info_final

,city
0,bangkok
1,buenos-aires
2,chiang-mai
3,taipei
4,canggu
...,...
1327,macaucordoba-spain
1328,george-town-cayman-islands
1329,victoria-seychelles
1330,macau


## country


In [366]:
# add the country
city_info_final['country'] = city_info_df['Country']
city_info_final

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,city,country
0,bangkok,Thailand
1,buenos-aires,Argentina
2,chiang-mai,Thailand
3,taipei,Taiwan
4,canggu,Indonesia
...,...,...
1327,macaucordoba-spain,China
1328,george-town-cayman-islands,Cayman Islands
1329,victoria-seychelles,Seychelles
1330,macau,China


## score

In [367]:
# add score as float and clean
city_info_final['score'] = [float(re.sub('/.+', '', str(x))) for x in city_info_df['Nomad Score']]
city_info_final

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,city,country,score
0,bangkok,Thailand,4.73
1,buenos-aires,Argentina,4.88
2,chiang-mai,Thailand,4.68
3,taipei,Taiwan,4.59
4,canggu,Indonesia,4.69
...,...,...,...
1327,macaucordoba-spain,China,3.56
1328,george-town-cayman-islands,Cayman Islands,2.00
1329,victoria-seychelles,Seychelles,2.46
1330,macau,China,3.56


## cost/month

In [368]:
# add cost as float and clean
city_info_final['cost/month'] = [float(re.sub(r'[\D+]', '', str(x))) for x in city_info_df['Cost']]
city_info_final

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,city,country,score,cost/month
0,bangkok,Thailand,4.73,1522.0
1,buenos-aires,Argentina,4.88,1026.0
2,chiang-mai,Thailand,4.68,1126.0
3,taipei,Taiwan,4.59,2120.0
4,canggu,Indonesia,4.69,1389.0
...,...,...,...,...
1327,macaucordoba-spain,China,3.56,2341.0
1328,george-town-cayman-islands,Cayman Islands,2.00,5423.0
1329,victoria-seychelles,Seychelles,2.46,4020.0
1330,macau,China,3.56,2341.0


## fun

In [369]:
city_info_final['fun'] = city_info_df['Fun']
city_info_final

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,city,country,score,cost/month,fun
0,bangkok,Thailand,4.73,1522.0,Good
1,buenos-aires,Argentina,4.88,1026.0,Good
2,chiang-mai,Thailand,4.68,1126.0,Good
3,taipei,Taiwan,4.59,2120.0,Okay
4,canggu,Indonesia,4.69,1389.0,Good
...,...,...,...,...,...
1327,macaucordoba-spain,China,3.56,2341.0,Great
1328,george-town-cayman-islands,Cayman Islands,2.00,5423.0,Bad
1329,victoria-seychelles,Seychelles,2.46,4020.0,Okay
1330,macau,China,3.56,2341.0,Great


## safety

In [370]:
city_info_final['safety'] = city_info_df['Safety']
# remove emoji
city_info_final['safety'] = city_info_final['safety'].apply(lambda x : remove_all_punctuation(x))

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [371]:
city_info_final[city_info_final['safety'] == '⚠️ Ongoing armed conflict']

,city,country,score,cost/month,fun,safety


## quality-of_life

In [372]:
city_info_final['quality_of_life'] = city_info_df['Quality of life']
city_info_final

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,city,country,score,cost/month,fun,safety,quality_of_life
0,bangkok,Thailand,4.73,1522.0,Good,Good,Good
1,buenos-aires,Argentina,4.88,1026.0,Good,Okay,Good
2,chiang-mai,Thailand,4.68,1126.0,Good,Great,Good
3,taipei,Taiwan,4.59,2120.0,Okay,Great,Great
4,canggu,Indonesia,4.69,1389.0,Good,Great,Good
...,...,...,...,...,...,...,...
1327,macaucordoba-spain,China,3.56,2341.0,Great,Good,Okay
1328,george-town-cayman-islands,Cayman Islands,2.00,5423.0,Bad,Good,Okay
1329,victoria-seychelles,Seychelles,2.46,4020.0,Okay,Good,Okay
1330,macau,China,3.56,2341.0,Great,Good,Okay


## walkability

In [373]:
city_info_final['walkability'] = city_info_df['Walkability']
city_info_final

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,city,country,score,cost/month,fun,safety,quality_of_life,walkability
0,bangkok,Thailand,4.73,1522.0,Good,Good,Good,Great
1,buenos-aires,Argentina,4.88,1026.0,Good,Okay,Good,Great
2,chiang-mai,Thailand,4.68,1126.0,Good,Great,Good,Great
3,taipei,Taiwan,4.59,2120.0,Okay,Great,Great,Great
4,canggu,Indonesia,4.69,1389.0,Good,Great,Good,Okay
...,...,...,...,...,...,...,...,...
1327,macaucordoba-spain,China,3.56,2341.0,Great,Good,Okay,Great
1328,george-town-cayman-islands,Cayman Islands,2.00,5423.0,Bad,Good,Okay,Bad
1329,victoria-seychelles,Seychelles,2.46,4020.0,Okay,Good,Okay,Bad
1330,macau,China,3.56,2341.0,Great,Good,Okay,Great


## happiness

In [374]:
city_info_final['happiness'] = city_info_df['Happiness']
city_info_final

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,city,country,score,cost/month,fun,safety,quality_of_life,walkability,happiness
0,bangkok,Thailand,4.73,1522.0,Good,Good,Good,Great,Good
1,buenos-aires,Argentina,4.88,1026.0,Good,Okay,Good,Great,Good
2,chiang-mai,Thailand,4.68,1126.0,Good,Great,Good,Great,Good
3,taipei,Taiwan,4.59,2120.0,Okay,Great,Great,Great,Good
4,canggu,Indonesia,4.69,1389.0,Good,Great,Good,Okay,Okay
...,...,...,...,...,...,...,...,...,...
1327,macaucordoba-spain,China,3.56,2341.0,Great,Good,Okay,Great,Okay
1328,george-town-cayman-islands,Cayman Islands,2.00,5423.0,Bad,Good,Okay,Bad,NaN
1329,victoria-seychelles,Seychelles,2.46,4020.0,Okay,Good,Okay,Bad,NaN
1330,macau,China,3.56,2341.0,Great,Good,Okay,Great,Okay


## all the other needed columns

In [375]:
city_info_final = (pd.concat([city_info_final, city_info_df[['Nightlife', 'Free WiFi in city',
                                                             'Friendly to foreigners', 'English speaking', 
                                                             'Average trip duration','Return rate','Hotel', 
                                                             'Airbnb']]],
                             axis=1))

In [376]:
city_info_final

,city,country,score,cost/month,fun,safety,quality_of_life,walkability,happiness,Nightlife,Free WiFi in city,Friendly to foreigners,English speaking,Average trip duration,Return rate,Hotel,Airbnb
0,bangkok,Thailand,4.73,1522.0,Good,Good,Good,Great,Good,Great,Great,Great,Okay,7 days,18% of visitors return,$31 / night,$51 / night
1,buenos-aires,Argentina,4.88,1026.0,Good,Okay,Good,Great,Good,Great,Okay,Good,Okay,25 days,16% of visitors return,$34 / night,$24 / night
2,chiang-mai,Thailand,4.68,1126.0,Good,Great,Good,Great,Good,Okay,Good,Good,Okay,28 days,15% of visitors return,$25 / night,$41 / night
3,taipei,Taiwan,4.59,2120.0,Okay,Great,Great,Great,Good,Good,Great,Great,Okay,15 days,15% of visitors return,$50 / night,$83 / night
4,canggu,Indonesia,4.69,1389.0,Good,Great,Good,Okay,Okay,Good,Good,Great,Good,28 days,17% of visitors return,$21 / night,$58 / night
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1327,macaucordoba-spain,China,3.56,2341.0,Great,Good,Okay,Great,Okay,Okay,Okay,Okay,Okay,2 days,10% of visitors return,$53 / night,$78 / night
1328,george-town-cayman-islands,Cayman Islands,2.00,5423.0,Bad,Good,Okay,Bad,NaN,Bad,Good,Great,Great,5 days,0% of visitors return,$184 / night,$83 / night
1329,victoria-seychelles,Seychelles,2.46,4020.0,Okay,Good,Okay,Bad,NaN,Okay,Okay,Good,Okay,6 days,0% of visitors return,$102 / night,$205 / night
1330,macau,China,3.56,2341.0,Great,Good,Okay,Great,Okay,Okay,Okay,Okay,Okay,2 days,10% of visitors return,$53 / night,$78 / night


## rename columns

In [377]:
city_info_final.columns

Index(['city', 'country', 'score', 'cost/month', 'fun', 'safety',
       'quality_of_life', 'walkability', 'happiness', 'Nightlife',
       'Free WiFi in city', 'Friendly to foreigners', 'English speaking',
       'Average trip duration', 'Return rate', 'Hotel', 'Airbnb'],
      dtype='object')

In [378]:
city_info_final.columns=['city', 'country', 'score', 'cost/month', 'fun', 'safety',
       'quality_of_life', 'walkability', 'happiness', 'nightlife',
       'wifi', 'friendly_to_foreigners', 'english',
       'avg_trip_length_days', 'return_rate_percent', 'hotel_price_night', 'airbnb_price_night']

In [379]:
city_info_final.columns

Index(['city', 'country', 'score', 'cost/month', 'fun', 'safety',
       'quality_of_life', 'walkability', 'happiness', 'nightlife', 'wifi',
       'friendly_to_foreigners', 'english', 'avg_trip_length_days',
       'return_rate_percent', 'hotel_price_night', 'airbnb_price_night'],
      dtype='object')

## clean more

In [380]:
city_info_final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1332 entries, 0 to 1331
Data columns (total 17 columns):
city                      1332 non-null object
country                   1332 non-null object
score                     1332 non-null float64
cost/month                1332 non-null float64
fun                       1327 non-null object
safety                    1332 non-null object
quality_of_life           1332 non-null object
walkability               1332 non-null object
happiness                 1255 non-null object
nightlife                 1330 non-null object
wifi                      1315 non-null object
friendly_to_foreigners    1036 non-null object
english                   1330 non-null object
avg_trip_length_days      860 non-null object
return_rate_percent       1332 non-null object
hotel_price_night         1328 non-null object
airbnb_price_night        1144 non-null object
dtypes: float64(2), object(15)
memory usage: 177.0+ KB


In [381]:
city_info_final.head()

,city,country,score,cost/month,fun,safety,quality_of_life,walkability,happiness,nightlife,wifi,friendly_to_foreigners,english,avg_trip_length_days,return_rate_percent,hotel_price_night,airbnb_price_night
0,bangkok,Thailand,4.73,1522.0,Good,Good,Good,Great,Good,Great,Great,Great,Okay,7 days,18% of visitors return,$31 / night,$51 / night
1,buenos-aires,Argentina,4.88,1026.0,Good,Okay,Good,Great,Good,Great,Okay,Good,Okay,25 days,16% of visitors return,$34 / night,$24 / night
2,chiang-mai,Thailand,4.68,1126.0,Good,Great,Good,Great,Good,Okay,Good,Good,Okay,28 days,15% of visitors return,$25 / night,$41 / night
3,taipei,Taiwan,4.59,2120.0,Okay,Great,Great,Great,Good,Good,Great,Great,Okay,15 days,15% of visitors return,$50 / night,$83 / night
4,canggu,Indonesia,4.69,1389.0,Good,Great,Good,Okay,Okay,Good,Good,Great,Good,28 days,17% of visitors return,$21 / night,$58 / night


### avg_trip_length_days

In [382]:
def keep_num(string):
    return float(re.sub(r'[\D+]', '', string))

In [383]:
city_info_final['avg_trip_length_days'].unique()

array(['7 days', '25 days', '28 days', '15 days', '14 days', '10 days',
       '29 days', '12 days', '8 days', '6 days', '5 days', '9 days',
       '16 days', '13 days', '4 days', '11 days', '18 days', '3 days',
       '20 days', '19 days', '21 days', nan, '2 days', '23 days',
       '22 days', '27 days', '17 days', '24 days', '31 days', '26 days',
       '30 days'], dtype=object)

In [384]:
# convert to int
city_info_final['avg_trip_length_days'] = [keep_num(x) if isinstance(x,str) else np.nan 
                                           for x in city_info_final['avg_trip_length_days']]

In [385]:
city_info_final['avg_trip_length_days']

0        7.0
1       25.0
2       28.0
3       15.0
4       28.0
        ... 
1327     2.0
1328     5.0
1329     6.0
1330     2.0
1331     7.0
Name: avg_trip_length_days, Length: 1332, dtype: float64

### return_rate_percent

In [386]:
# convert to int
city_info_final['return_rate_percent'] = [keep_num(x) if isinstance(x,str) else np.nan 
                                           for x in city_info_final['return_rate_percent']]
city_info_final['return_rate_percent']

0       18.0
1       16.0
2       15.0
3       15.0
4       17.0
        ... 
1327    10.0
1328     0.0
1329     0.0
1330    10.0
1331     0.0
Name: return_rate_percent, Length: 1332, dtype: float64

### hotel_price_night, airbnb_price_night

In [387]:
# convert to int
city_info_final['hotel_price_night'] = [keep_num(x) if isinstance(x,str) else np.nan 
                                           for x in city_info_final['hotel_price_night']]

In [388]:
# convert to int
city_info_final['airbnb_price_night'] = [keep_num(x) if isinstance(x,str) else np.nan 
                                           for x in city_info_final['airbnb_price_night']]

# Cleaned Data

In [389]:
city_info_final

,city,country,score,cost/month,fun,safety,quality_of_life,walkability,happiness,nightlife,wifi,friendly_to_foreigners,english,avg_trip_length_days,return_rate_percent,hotel_price_night,airbnb_price_night
0,bangkok,Thailand,4.73,1522.0,Good,Good,Good,Great,Good,Great,Great,Great,Okay,7.0,18.0,31.0,51.0
1,buenos-aires,Argentina,4.88,1026.0,Good,Okay,Good,Great,Good,Great,Okay,Good,Okay,25.0,16.0,34.0,24.0
2,chiang-mai,Thailand,4.68,1126.0,Good,Great,Good,Great,Good,Okay,Good,Good,Okay,28.0,15.0,25.0,41.0
3,taipei,Taiwan,4.59,2120.0,Okay,Great,Great,Great,Good,Good,Great,Great,Okay,15.0,15.0,50.0,83.0
4,canggu,Indonesia,4.69,1389.0,Good,Great,Good,Okay,Okay,Good,Good,Great,Good,28.0,17.0,21.0,58.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1327,macaucordoba-spain,China,3.56,2341.0,Great,Good,Okay,Great,Okay,Okay,Okay,Okay,Okay,2.0,10.0,53.0,78.0
1328,george-town-cayman-islands,Cayman Islands,2.00,5423.0,Bad,Good,Okay,Bad,NaN,Bad,Good,Great,Great,5.0,0.0,184.0,83.0
1329,victoria-seychelles,Seychelles,2.46,4020.0,Okay,Good,Okay,Bad,NaN,Okay,Okay,Good,Okay,6.0,0.0,102.0,205.0
1330,macau,China,3.56,2341.0,Great,Good,Okay,Great,Okay,Okay,Okay,Okay,Okay,2.0,10.0,53.0,78.0


In [390]:
city_info_final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1332 entries, 0 to 1331
Data columns (total 17 columns):
city                      1332 non-null object
country                   1332 non-null object
score                     1332 non-null float64
cost/month                1332 non-null float64
fun                       1327 non-null object
safety                    1332 non-null object
quality_of_life           1332 non-null object
walkability               1332 non-null object
happiness                 1255 non-null object
nightlife                 1330 non-null object
wifi                      1315 non-null object
friendly_to_foreigners    1036 non-null object
english                   1330 non-null object
avg_trip_length_days      860 non-null float64
return_rate_percent       1332 non-null float64
hotel_price_night         1328 non-null float64
airbnb_price_night        1144 non-null float64
dtypes: float64(6), object(11)
memory usage: 177.0+ KB


# Add user info

In [392]:
city_info_final['visitors'] = pd.DataFrame(list([people_info_dict.values()])).transpose()

## Remove cities with no visitors

In [393]:
city_info_final[city_info_final['city'] == 'cancun']['visitors'].str.len()

46    90
Name: visitors, dtype: int64

# Add city + country

In [394]:
cities = list(city_info_final['city'])
countries = list(city_info_final['country'])

In [395]:
city_and_country = []
for i in range(0, len(cities)):
    city_and_country.append(remove_all_punctuation(cities[i] + ' ' + countries[i].lower()))

In [396]:
city_info_final.insert(2, 'city_and_country', city_and_country)

In [397]:
city_info_final

,city,country,city_and_country,score,cost/month,fun,safety,quality_of_life,walkability,happiness,nightlife,wifi,friendly_to_foreigners,english,avg_trip_length_days,return_rate_percent,hotel_price_night,airbnb_price_night,visitors
0,bangkok,Thailand,bangkok thailand,4.73,1522.0,Good,Good,Good,Great,Good,Great,Great,Great,Okay,7.0,18.0,31.0,51.0,"[/@remyp, /@manas, /@timrael, /@dimqen, /@dani..."
1,buenos-aires,Argentina,buenos aires argentina,4.88,1026.0,Good,Okay,Good,Great,Good,Great,Okay,Good,Okay,25.0,16.0,34.0,24.0,"[/@krausefx, /@aczuleta, /@alexanderjoo, /@sil..."
2,chiang-mai,Thailand,chiang mai thailand,4.68,1126.0,Good,Great,Good,Great,Good,Okay,Good,Good,Okay,28.0,15.0,25.0,41.0,"[/@meedamian, /@zapperen, /@john, /@kymellis, ..."
3,taipei,Taiwan,taipei taiwan,4.59,2120.0,Okay,Great,Great,Great,Good,Good,Great,Great,Okay,15.0,15.0,50.0,83.0,"[/@onur, /@coffeejunk, /@justinl, /@majkp23, /..."
4,canggu,Indonesia,canggu indonesia,4.69,1389.0,Good,Great,Good,Okay,Okay,Good,Good,Great,Good,28.0,17.0,21.0,58.0,"[/@tris, /@joytravels, /@guar47, /@mariebriand..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1327,macaucordoba-spain,China,macaucordoba spain china,3.56,2341.0,Great,Good,Okay,Great,Okay,Okay,Okay,Okay,Okay,2.0,10.0,53.0,78.0,"[/@silviina, /@nateberkopec, /@angel, /@gvrizz..."
1328,george-town-cayman-islands,Cayman Islands,george town cayman islands cayman islands,2.00,5423.0,Bad,Good,Okay,Bad,NaN,Bad,Good,Great,Great,5.0,0.0,184.0,83.0,"[/@quintinjadam, /@markwlewis, /@maximus, /@je..."
1329,victoria-seychelles,Seychelles,victoria seychelles seychelles,2.46,4020.0,Okay,Good,Okay,Bad,NaN,Okay,Okay,Good,Okay,6.0,0.0,102.0,205.0,"[/@michaelchiche, /@kmiloloza, /@travpreneur, ..."
1330,macau,China,macau china,3.56,2341.0,Great,Good,Okay,Great,Okay,Okay,Okay,Okay,Okay,2.0,10.0,53.0,78.0,"[/@silviina, /@nateberkopec, /@angel, /@gvrizz..."


# Save to pickle

In [398]:
# save the final dataframe
with open(f'city_info_final.pkl', 'wb') as picklefile:
    pickle.dump(city_info_final, picklefile)

# Scrape WikiVoyage

In [399]:
url = 'https://en.wikivoyage.org/w/index.php?sort=relevance&search=bangkok&title=Special:Search&profile=advanced&fulltext=1&advancedSearch-current=%7B%7D&ns0=1'

In [400]:
# path to the chromedriver executable
chromedriver = "/Applications/chromedriver"
os.environ["webdriver.chrome.driver"] = chromedriver

driver = webdriver.Chrome(chromedriver)
driver.get(url)

# Wait for page to load
time.sleep(3) # random time based onthe length of the text

In [401]:
wiki_links = []

for city in tqdm(cities):
    try:
        # Search for the city we want
        search_box = driver.find_element_by_xpath("//input[@type='search']")

        # clear the current search
        search_box.clear()

        # input city
        search_box.send_keys(city) 

        # click enter
        search_box.send_keys(Keys.RETURN)

        # wait for element to appear, then hover it
        wait = WebDriverWait(driver, 10)
        men_menu = wait.until(expected_conditions.visibility_of_element_located(
            (By.XPATH, "//div[@class='searchresults']")))
        ActionChains(driver).move_to_element(men_menu).perform()
        
        
        time.sleep(1)
        
        
        
        # get first link, which should be the wikitravel link
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        wiki_link = soup.find('div', class_='mw-search-result-heading').find('a').get('href')
        wiki_links.append(wiki_link)


        # append to pickle
        with open('wiki_travel_links.pkl', 'wb') as picklefile:
            pickle.dump(wiki_links, picklefile)
        print(wiki_link)
    except:
        wiki_links.append('SKIPPED')
        
        # append to pickle
        with open('wiki_travel_links.pkl', 'wb') as picklefile:
            pickle.dump(wiki_links, picklefile)

        print('Exception', city)

## Open Wiki Voyage Links 

In [402]:
# open links
with open("wiki_travel_links.pkl", 'rb') as picklefile: 
    wiki_travel_links = pickle.load(picklefile)

In [403]:
wiki_links_df = pd.DataFrame(cities, wiki_travel_links).reset_index()
wiki_links_df.columns=['link', 'city']

# Remove cities with no visitors

In [415]:
# drop cities with no visitors
city_info_final = city_info_final[city_info_final['visitors'].str.len() > 0]

In [416]:
city_info_final.reset_index(inplace=True)

In [417]:
city_info_final

,index,city,country,city_and_country,score,cost/month,fun,safety,quality_of_life,walkability,happiness,nightlife,wifi,friendly_to_foreigners,english,avg_trip_length_days,return_rate_percent,hotel_price_night,airbnb_price_night,visitors
0,0,bangkok,Thailand,bangkok thailand,4.73,1522.0,Good,Good,Good,Great,Good,Great,Great,Great,Okay,7.0,18.0,31.0,51.0,"[/@remyp, /@manas, /@timrael, /@dimqen, /@dani..."
1,1,buenos-aires,Argentina,buenos aires argentina,4.88,1026.0,Good,Okay,Good,Great,Good,Great,Okay,Good,Okay,25.0,16.0,34.0,24.0,"[/@krausefx, /@aczuleta, /@alexanderjoo, /@sil..."
2,2,chiang-mai,Thailand,chiang mai thailand,4.68,1126.0,Good,Great,Good,Great,Good,Okay,Good,Good,Okay,28.0,15.0,25.0,41.0,"[/@meedamian, /@zapperen, /@john, /@kymellis, ..."
3,3,taipei,Taiwan,taipei taiwan,4.59,2120.0,Okay,Great,Great,Great,Good,Good,Great,Great,Okay,15.0,15.0,50.0,83.0,"[/@onur, /@coffeejunk, /@justinl, /@majkp23, /..."
4,4,canggu,Indonesia,canggu indonesia,4.69,1389.0,Good,Great,Good,Okay,Okay,Good,Good,Great,Good,28.0,17.0,21.0,58.0,"[/@tris, /@joytravels, /@guar47, /@mariebriand..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1014,1327,macaucordoba-spain,China,macaucordoba spain china,3.56,2341.0,Great,Good,Okay,Great,Okay,Okay,Okay,Okay,Okay,2.0,10.0,53.0,78.0,"[/@silviina, /@nateberkopec, /@angel, /@gvrizz..."
1015,1328,george-town-cayman-islands,Cayman Islands,george town cayman islands cayman islands,2.00,5423.0,Bad,Good,Okay,Bad,NaN,Bad,Good,Great,Great,5.0,0.0,184.0,83.0,"[/@quintinjadam, /@markwlewis, /@maximus, /@je..."
1016,1329,victoria-seychelles,Seychelles,victoria seychelles seychelles,2.46,4020.0,Okay,Good,Okay,Bad,NaN,Okay,Okay,Good,Okay,6.0,0.0,102.0,205.0,"[/@michaelchiche, /@kmiloloza, /@travpreneur, ..."
1017,1330,macau,China,macau china,3.56,2341.0,Great,Good,Okay,Great,Okay,Okay,Okay,Okay,Okay,2.0,10.0,53.0,78.0,"[/@silviina, /@nateberkopec, /@angel, /@gvrizz..."


# Merge links with final dataframe

In [419]:
city_info_final = (city_info_final.merge(wiki_links_df, left_on='city', right_on='city')
                   .reset_index()
                   .drop(['level_0', 'index'], axis=1))

In [420]:
city_info_final.sort_values(by='score', ascending=False, inplace=True)

In [421]:
city_info_final.reset_index(inplace=True, drop=True)

In [422]:
city_info_final

,city,country,city_and_country,score,cost/month,fun,safety,quality_of_life,walkability,happiness,nightlife,wifi,friendly_to_foreigners,english,avg_trip_length_days,return_rate_percent,hotel_price_night,airbnb_price_night,visitors,link
0,buenos-aires,Argentina,buenos aires argentina,4.88,1026.0,Good,Okay,Good,Great,Good,Great,Okay,Good,Okay,25.0,16.0,34.0,24.0,"[/@krausefx, /@aczuleta, /@alexanderjoo, /@sil...",/wiki/Buenos_Aires
1,bangkok,Thailand,bangkok thailand,4.73,1522.0,Good,Good,Good,Great,Good,Great,Great,Great,Okay,7.0,18.0,31.0,51.0,"[/@remyp, /@manas, /@timrael, /@dimqen, /@dani...",/wiki/Bangkok
2,mexico-city,Mexico,mexico city mexico,4.72,1493.0,Good,Okay,Good,Great,Good,Good,Okay,Good,Okay,14.0,14.0,30.0,31.0,"[/@rohit, /@nadiaronquillo, /@evelienal, /@bri...",/wiki/Mexico_City/Santa_Fe
3,canggu,Indonesia,canggu indonesia,4.69,1389.0,Good,Great,Good,Okay,Okay,Good,Good,Great,Good,28.0,17.0,21.0,58.0,"[/@tris, /@joytravels, /@guar47, /@mariebriand...",/wiki/Canggu
4,chiang-mai,Thailand,chiang mai thailand,4.68,1126.0,Good,Great,Good,Great,Good,Okay,Good,Good,Okay,28.0,15.0,25.0,41.0,"[/@meedamian, /@zapperen, /@john, /@kymellis, ...",/wiki/Chiang_Mai
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1006,northampton,United Kingdom,northampton united kingdom,1.90,5216.0,Bad,Good,Okay,Great,Good,Bad,Okay,Good,Great,4.0,0.0,171.0,NaN,"[/@iainc, /@john]",/wiki/Northampton
1007,lincoln,United States,lincoln united states,1.90,5320.0,Okay,Good,Okay,Okay,Good,Okay,Good,NaN,Great,7.0,0.0,233.0,NaN,"[/@cpitkin, /@residualdata, /@graham, /@brody,...",/wiki/Lincoln
1008,noumea,New Caledonia,noumea new caledonia,1.89,5377.0,Okay,Good,Okay,Bad,NaN,Bad,Bad,Bad,Bad,NaN,0.0,80.0,50.0,[/@shadiosta],/wiki/Noum%C3%A9a
1009,ashland,United States,ashland united states,1.88,5231.0,Okay,Good,Okay,Bad,Good,Okay,Good,Okay,Great,26.0,0.0,158.0,NaN,"[/@camposped, /@adamnowek, /@mjmeidinger]",/wiki/Ashland


# Final DataFrame

In [423]:
# save the final dataframe
with open(f'city_info_final.pkl', 'wb') as picklefile:
    pickle.dump(city_info_final, picklefile)

# Get WikiVoyage info

In [3]:
wiki_voyage_content = []
for link in tqdm(city_info_final['link']):
    city_content = []
    link = 'https://en.wikivoyage.org' + link
    
    response = requests.get(link)
    page = response.text
    soup = BeautifulSoup(page, "html.parser")

    content = soup.find(class_='mw-content-ltr')
    for para in content.find_all('p'):
        city_content.append(para.text)
        
    wiki_voyage_content.append(' '.join(city_content))
    
    # save the final dataframe
    with open(f'wiki_voyage_content.pkl', 'wb') as picklefile:
        pickle.dump(wiki_voyage_content, picklefile)